In [ ]:
!pip install tensorflow==2.14 keras_cv

In [ ]:
!pip uninstall scikit-learn -y

!pip install -U scikit-learn

In [ ]:
# Fix randomness and hide warnings
seed = 42
input_shape = (96, 96, 3)

import os
os.environ['PYTHONHASHSEED'] = str(seed)

import numpy as np
import math
np.random.seed(seed)
import pandas as pd

# Import tensorflow
import tensorflow as tf
from tensorflow import keras as tfk
from tensorflow.keras.applications.convnext import preprocess_input
from tensorflow.keras import layers as tfkl
from tensorflow.keras import regularizers
import keras_cv

l2_reg = 0.001

tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

# Import other libraries
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, KFold
from sklearn.utils.class_weight import compute_class_weight

# ConvNeXt Model

In [ ]:
convnext = tfk.applications.ConvNeXtLarge(
      input_shape = input_shape,
      include_top = False,
      pooling='avg'
  )

convnext.trainable = False

In [ ]:
inputs = tfk.Input(shape=input_shape)

x = tf.keras.Sequential([
    tfkl.RandomFlip("horizontal"),
    tfkl.RandomTranslation(0.05, 0.05),
    tfkl.RandomRotation(0.07),
    tfkl.RandomZoom(0.03),
    #tfkl.RandomFlip("vertical"),
])(inputs)

x = convnext(x)

x = tfkl.Dense(
    75,
    activation='selu',
    kernel_regularizer=tfk.regularizers.l2(1e-5),
    name="c_dense0")(x)
x = tfkl.Dropout(0.3, name="drop0")(x)


outputs = tfkl.Dense(2, activation='softmax', name="c_output")(x)

model = tfk.Model(inputs=inputs, outputs=outputs, name='model')

# Define the optimizer with the schedule (Adam for custom weight decay)
optimizer = tfk.optimizers.AdamW()

early_stopping = tfk.callbacks.EarlyStopping(
    monitor='val_accuracy',          # Monitor validation accuracy
    mode='max',                  # Mode 'max' because we want to maximize validation accuracy
    patience=13,                  # Number of epochs with no improvement after which training will be stopped
    restore_best_weights=True)   # Restore model weights from the epoch with the best value of the monitored quantity

reduce_lr_on_plateau = tfk.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.2,
    patience=5
    )

callbacks = [early_stopping, reduce_lr_on_plateau]

model.compile(
    loss=tfk.losses.CategoricalCrossentropy(label_smoothing=0.1),
    optimizer=optimizer,
    metrics=['accuracy']
)
model.summary()

# Data Import

In [ ]:
import gc
gc.collect()

In [ ]:
def downsample_dataset(images, labels):
    # Count the instances in each class
    class_counts = np.bincount(labels)
    min_class_count = np.min(class_counts)

    # Indices for each class
    indices_class_0 = np.where(labels == 0)[0]
    indices_class_1 = np.where(labels == 1)[0]

    # Randomly select 'min_class_count' indices from each class
    sampled_indices_class_0 = np.random.choice(indices_class_0, min_class_count, replace=False)
    sampled_indices_class_1 = np.random.choice(indices_class_1, min_class_count, replace=False)

    # Combine and shuffle
    final_indices = np.concatenate((sampled_indices_class_0, sampled_indices_class_1))
    np.random.shuffle(final_indices)

    # Subset the images and labels
    downsampled_images = images[final_indices]
    downsampled_labels = labels[final_indices]

    return downsampled_images, downsampled_labels

In [ ]:
data = np.load('../input/plants/augmented_train_mix-2.npz', allow_pickle=True)
X_train, y_train = data["data"], data["labels"]

data = np.load('../input/plants/val_unbalanced_mix-2.npz', allow_pickle=True)
X_val, y_val = data["data"], data["labels"]

In [ ]:
#imgs, labels = downsample_dataset(imgs, labels)
#X_train, X_val, y_train, y_val = train_test_split(imgs, labels, test_size=.2)

X_train.shape, y_train.shape

In [ ]:
_, (n_healthy, n_unhealthy) = np.unique(y_train, return_counts=True)
(n_healthy, n_unhealthy, ((n_healthy)/len(y_train))*100, ((n_unhealthy)/len(y_train))*100)

In [ ]:
_, (n_healthy, n_unhealthy) = np.unique(y_val, return_counts=True)
(n_healthy, n_unhealthy, ((n_healthy)/len(y_val))*100, ((n_unhealthy)/len(y_val))*100)

In [ ]:
# Define CutMix and MixUp
cut_mix = keras_cv.layers.CutMix()
mix_up = keras_cv.layers.MixUp()

def apply_cutmix_or_mixup_augmentations(images, labels):
    # Initialize containers for augmented images and labels
    aug_imgs = []
    aug_labels = []

    for img, lbl in zip(images, labels):
        # Randomly choose between CutMix and MixUp for each image
        if np.random.rand() < 0.5:  # 50% chance for CutMix
            aug_tmp = cut_mix({'images': np.expand_dims(img, 0),
                               'labels': np.expand_dims(lbl, 0)},
                               training=True)
        else:  # 50% chance for MixUp
            aug_tmp = mix_up({'images': np.expand_dims(img, 0),
                              'labels': np.expand_dims(lbl, 0)},
                              training=True)

        # Append the augmented image and label
        aug_imgs.append(aug_tmp['images'][0].numpy())
        aug_labels.append(aug_tmp['labels'][0].numpy())

    # Convert lists to numpy arrays
    aug_imgs = np.array(aug_imgs)
    aug_labels = np.array(aug_labels)

    return aug_imgs, aug_labels


In [ ]:
y_train = tf.cast(tf.one_hot(y_train, 2), tf.float32)
y_val = tf.cast(tf.one_hot(y_val, 2), tf.float32)
y_train, y_val

In [ ]:
X_train, y_train = apply_cutmix_or_mixup_augmentations(X_train, y_train)

In [ ]:
y_val = y_val.numpy()

## Training

In [ ]:
history = model.fit(
    X_train,
    y_train,
    validation_data=(X_val, y_val),
    epochs = 90,
    callbacks = callbacks,
    batch_size=128
).history

In [ ]:
# Plot the training
plt.figure(figsize=(15,5))
plt.plot(history['loss'], alpha=.3, color='#4D61E2', linestyle='--')
plt.plot(history['val_loss'], label='CNN with Augmentation', alpha=.8, color='#4D61E2')
plt.legend(loc='upper left')
plt.title('Binary Crossentropy')
plt.grid(alpha=.3)

plt.figure(figsize=(15,5))
plt.plot(history['accuracy'], alpha=.3, color='#4D61E2', linestyle='--')
plt.plot(history['val_accuracy'], label='CNN with Augmentation', alpha=.8, color='#4D61E2')
plt.legend(loc='upper left')
plt.title('Accuracy')
plt.grid(alpha=.3)

plt.show()

# Saving

In [ ]:
model.save("/kaggle/working/Convnext_mixup")

In [ ]:
del model

In [ ]:
gc.collect()

# Fine Tune


In [ ]:
model_convTL = tfk.models.load_model('/kaggle/working/Convnext_mixup')

In [ ]:
model_convTL.get_layer("convnext_large").trainable = True

In [ ]:
model_convTL.get_layer("c_dense0").kernel_regularizer = tfk.regularizers.L1L2(l1=1e-4, l2=5e-4)
model_convTL.get_layer("c_output").kernel_regularizer = tfk.regularizers.L2(1e-5)

In [ ]:
# Define a learning rate schedule
lr_schedule = tfk.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-5,  # Start with this learning rate
    decay_steps=900,           # After how many steps to apply decay
    decay_rate=0.9,              # Decay rate
    staircase=True)              # If True, learning rate is reduced at discrete intervals

# Define the optimizer with the schedule
optimizer = tfk.optimizers.Adam(learning_rate=lr_schedule)

def compile_model(model):
    model.compile(
        loss=tfk.losses.BinaryCrossentropy(),
        optimizer=optimizer,
        metrics=['accuracy']
    )

compile_model(model_convTL)
model_convTL.summary()

In [ ]:
early_stopping = tfk.callbacks.EarlyStopping(
    monitor='val_accuracy',
    mode='max',
    patience=4,
    restore_best_weights=True)

reduce_lr_on_plateau = tfk.callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.7,
    patience=5
    )

callbacks = [early_stopping, reduce_lr_on_plateau]

In [ ]:
def unfreeze_model_layers(model, N):
    for layer in model.get_layer('convnext_large').layers[:-N]:
        layer.trainable = False
    for layer in model.get_layer('convnext_large').layers[-N:]:
        layer.trainable = True

    print(f"Unlocked: {sum(layer.trainable for layer in model.get_layer('convnext_large').layers)}/{len(model.get_layer('convnext_large').layers)}")
    compile_model(model)

In [ ]:
unfreeze_model_layers(model_convTL, 176)
history = model_convTL.fit(
    X_train,
    y_train,
    validation_data=(X_val, y_val),
    epochs = 90,
    callbacks = callbacks,
    batch_size=64
).history

In [ ]:
# Plot the training
plt.figure(figsize=(15,5))
plt.plot(history['loss'], alpha=.3, color='#4D61E2', linestyle='--')
plt.plot(history['val_loss'], label='CNN with Augmentation', alpha=.8, color='#4D61E2')
plt.legend(loc='upper left')
plt.title('Binary Crossentropy')
plt.grid(alpha=.3)

plt.figure(figsize=(15,5))
plt.plot(history['accuracy'], alpha=.3, color='#4D61E2', linestyle='--')
plt.plot(history['val_accuracy'], label='CNN with Augmentation', alpha=.8, color='#4D61E2')
plt.legend(loc='upper left')
plt.title('Accuracy')
plt.grid(alpha=.3)

plt.show()

In [ ]:
data = np.load('../input/plants/clean_data.npz', allow_pickle=True)
X_test, y_test = data["data"], data["labels"]
y_test = (y_test == "unhealthy").astype("int")
y_test = tf.cast(tf.one_hot(y_test, 2), tf.float32)
model_convTL.evaluate(X_test, y_test)

In [ ]:
model_convTL.save("/kaggle/working/Convnext_mixup_ds_finetuned_09471")

In [ ]:
!zip -r Model.zip /kaggle/working/Convnext_mixup_ds_finetuned_09471

In [ ]:
from IPython.display import FileLink
FileLink(r'Model.zip')